利用BM25算法对已有QA对分等级（简单、中等、困难），模型：豆包

In [ ]:
import json
import requests
import time
from tqdm import tqdm
from opencc import OpenCC
import jieba

API_KEY = "d601827e-cd0d-41e2-9c0a-32e424214e6a" 
API_URL = "https://ark.cn-beijing.volces.com/api/v3/chat/completions"
HEADERS = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}

cc = OpenCC('t2s')

def tokenize(text):
    """中文分词函数"""
    return list(jieba.cut(text))

def build_prompt(question, answer):
    return f"""
你是一位保险领域的资深专家，请根据以下问答对的复杂程度和详细程度，将其划分为"简单"、"中等"或"困难"三个等级。

### 划分标准：
1. 简单：问题直接明确，答案简短，涉及基本保险概念或简单流程
   - 示例：什么是寿险？
   - 特征：单概念、单步回答

2. 中等：问题需要一定专业知识理解，答案包含多个要点或步骤
   - 示例：重疾险的理赔流程是什么？
   - 特征：多步骤、2-3个相关概念

3. 困难：问题复杂或涉及专业术语，答案需要详细解释或多方面考虑
   - 示例：如何评估不同投资连结保险产品的风险收益特征？
   - 特征：多概念交叉、需要推理分析

### 评估维度：
1. 问题复杂度：问题的专业性和抽象程度
2. 答案详细度：答案的深度、广度和技术细节
3. 概念数量：涉及的专业概念数量
4. 推理需求：是否需要多步推理或案例分析

### 待评估QA对：
问题：{question}
回答：{answer}

### 输出要求：
请严格按以下JSON格式回应：
{{
    "difficulty": "简单|中等|困难",
    "reason": "分级理由说明"
}}
"""

def classify_qa_pair(question, answer):
    prompt = build_prompt(question, answer)
    data = {
        "model": "doubao-1-5-thinking-pro-250415", 
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.3,
        "response_format": {"type": "json_object"} 
    }
    
    try:
        response = requests.post(API_URL, headers=HEADERS, json=data, timeout=10)
        response.raise_for_status() 
        
        result = response.json()
       
        if "choices" in result:
            content = result["choices"][0]["message"]["content"]
        elif "result" in result:  
            content = result["result"]
        else:
            content = json.dumps({"difficulty": "未知", "reason": "API响应格式不符"})
            
        try:
            json.loads(content)  
            return content
        except json.JSONDecodeError:
            return json.dumps({"difficulty": "未知", "reason": "API返回非JSON格式"})
            
    except requests.exceptions.RequestException as e:
        print(f"API请求错误: {str(e)}")
        return json.dumps({"difficulty": "未知", "reason": f"API请求错误: {str(e)}"})
    except Exception as e:
        print(f"处理异常: {str(e)}")
        return json.dumps({"difficulty": "未知", "reason": f"处理异常: {str(e)}"})

def process_testset(qa_file, output_file):
    with open(qa_file, "r", encoding="utf-8") as f:
        qa_list = json.load(f)

    results = []
    stats = {"简单": 0, "中等": 0, "困难": 0, "未知": 0}
    
    print("\n开始处理QA对难度分级...\n")
    
    for item in tqdm(qa_list, desc="处理进度", unit="对"):
        question = item.get("question", "")
        answer = item.get("answer", "")
        
        try:
            classification = classify_qa_pair(question, answer)
            classification_data = json.loads(classification)
            
            item["difficulty"] = classification_data.get("difficulty", "未知")
            item["difficulty_reason"] = classification_data.get("reason", "")
            
            stats[item["difficulty"]] += 1
        except json.JSONDecodeError:
            item["difficulty"] = "未知"
            item["difficulty_reason"] = "响应解析失败"
            stats["未知"] += 1
        
        results.append(item)
        time.sleep(1)  

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    
    print("\n分级统计结果:")
    for level, count in stats.items():
        print(f"{level}: {count}个")
    print("\n所有问答难度分级已完成，结果保存在：", output_file)

if __name__ == "__main__":
    process_testset(
        r"D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA4.json",    
        r"D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA_level（QA对）.json"          
    )


开始处理QA对难度分级...



处理进度:  36%|███▌      | 79/222 [09:16<19:43,  8.27s/对]

API请求错误: HTTPSConnectionPool(host='ark.cn-beijing.volces.com', port=443): Read timed out. (read timeout=10)


处理进度:  37%|███▋      | 82/222 [09:43<19:44,  8.46s/对]

API请求错误: HTTPSConnectionPool(host='ark.cn-beijing.volces.com', port=443): Read timed out. (read timeout=10)


...

API请求错误: HTTPSConnectionPool(host='ark.cn-beijing.volces.com', port=443): Read timed out. (read timeout=10)


处理进度:  90%|████████▉ | 199/222 [23:39<03:28,  9.07s/对]

API请求错误: HTTPSConnectionPool(host='ark.cn-beijing.volces.com', port=443): Read timed out. (read timeout=10)


处理进度: 100%|██████████| 222/222 [26:31<00:00,  7.17s/对]


分级统计结果:
简单: 65个
中等: 145个
困难: 2个
未知: 10个

所有问答难度分级已完成，结果保存在： D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA_level（QA对）.json


丰富提示词，再进行评分（结果：简单变多，未知变多，效果下降），模型：豆包

In [ ]:
import json
import requests
import time
from tqdm import tqdm
from opencc import OpenCC
import jieba

API_KEY = "d601827e-cd0d-41e2-9c0a-32e424214e6a"  
API_URL = "https://ark.cn-beijing.volces.com/api/v3/chat/completions"
HEADERS = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}

cc = OpenCC('t2s')

def tokenize(text):
    """中文分词函数"""
    return list(jieba.cut(text))

def build_prompt(question, answer):
    return f"""
你是一位保险领域的资深专家，请根据以下问答对的复杂程度和详细程度，将其划分为"简单"、"中等"或"困难"三个等级。

### 划分标准：
1. 简单：问题直接明确，答案简短，涉及基本保险概念或简单流程，无需复杂推理与专业知识
   - 示例：问题 “使用宏利信用卡支付保费有哪些限制条件？”，答案清晰列出几个条件，表述直接。
   - 特征：答案信息能直接从文档中明确找到，无需复杂推理。问题不涉及多个概念的综合理解，仅针对单一明确信息。答案无需额外的背景知识或行业经验即可理解。答案内容简短，通常为一句话或几个简单句子就能完整回答。属于常见的基础问题，如基本的操作步骤、常见政策规定等，不涉及特殊情况或复杂场景。

2. 中等：问题有一定复杂度，答案需整合多处信息并进行一定推理，涉及常见保险场景与概念
   - 示例：问题 “如果保单账户余额不足导致无法扣除月费，会有什么后果？”，答案需考虑宽限期等情况并详细说明。
   - 特征：答案需要在文档中综合多处信息才能得出。问题涉及一定的概念理解，但不是特别专业或生僻。需要一定的逻辑推理来整合信息回答问题。答案包含多个步骤或要点，但结构相对清晰。可能需要对某些条款或概念进行一定的解释。问题涉及一定的场景变化或条件限制，但不是特别复杂。不是最常见的问题，但也不是非常罕见的情况。

3. 困难：问题复杂，涉及专业保险术语、特殊场景和综合分析，答案需大量专业知识和深入解读
   - 示例：问题 “如果保险的次位受益人是未成年人，为什么不能由首位受益人担任其信托人？”，答案需解释背后的逻辑矛盾。
   - 特征：答案难以从文档中直接获取，需要深入分析和解读大量信息。问题涉及专业的保险术语、复杂的条款细则。需要丰富的行业背景知识和经验才能准确理解问题和回答。答案内容冗长，包含多个复杂的步骤、条件和解释。可能涉及多种情况的分析和对比。问题描述的场景复杂，包含多个变量和特殊情况。属于不常见的问题，需要考虑多种因素的综合影响。

### 评估维度：
1. 信息获取的难易程度
2. 问题涉及概念的复杂程度
3. 答案的复杂度和长度
4. 问题场景的复杂程度

### 待评估QA对：
问题：{question}
回答：{answer}

### 输出要求：
请严格按以下JSON格式回应：
{{
    "difficulty": "简单|中等|困难",
    "reason": "分级理由说明"
}}
"""

def classify_qa_pair(question, answer):
    prompt = build_prompt(question, answer)
    data = {
        "model": "doubao-1-5-thinking-pro-250415", 
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.3,
        "response_format": {"type": "json_object"}  
    }
    
    try:
        response = requests.post(API_URL, headers=HEADERS, json=data, timeout=10)
        response.raise_for_status()  
        
        result = response.json()
        
        if "choices" in result:
            content = result["choices"][0]["message"]["content"]
        elif "result" in result:  
            content = result["result"]
        else:
            content = json.dumps({"difficulty": "未知", "reason": "API响应格式不符"})
            
        try:
            json.loads(content) 
            return content
        except json.JSONDecodeError:
            return json.dumps({"difficulty": "未知", "reason": "API返回非JSON格式"})
            
    except requests.exceptions.RequestException as e:
        print(f"API请求错误: {str(e)}")
        return json.dumps({"difficulty": "未知", "reason": f"API请求错误: {str(e)}"})
    except Exception as e:
        print(f"处理异常: {str(e)}")
        return json.dumps({"difficulty": "未知", "reason": f"处理异常: {str(e)}"})

def process_testset(qa_file, output_file):
    with open(qa_file, "r", encoding="utf-8") as f:
        qa_list = json.load(f)

    results = []
    stats = {"简单": 0, "中等": 0, "困难": 0, "未知": 0}
    
    print("\n开始处理QA对难度分级...\n")
    
    for item in tqdm(qa_list, desc="处理进度", unit="对"):
        question = item.get("question", "")
        answer = item.get("answer", "")
        
        try:
            classification = classify_qa_pair(question, answer)
            classification_data = json.loads(classification)
            
            item["difficulty"] = classification_data.get("difficulty", "未知")
            item["difficulty_reason"] = classification_data.get("reason", "")
            
            stats[item["difficulty"]] += 1
        except json.JSONDecodeError:
            item["difficulty"] = "未知"
            item["difficulty_reason"] = "响应解析失败"
            stats["未知"] += 1
        
        results.append(item)
        time.sleep(1)  

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    
    print("\n分级统计结果:")
    for level, count in stats.items():
        print(f"{level}: {count}个")
    print("\n所有问答难度分级已完成，结果保存在：", output_file)

if __name__ == "__main__":
    process_testset(
        r"D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA4.json",    
        r"D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA_level（QA对）(2).json"          
    )


开始处理QA对难度分级...



处理进度:   0%|          | 1/222 [00:07<27:03,  7.35s/对]

API请求错误: HTTPSConnectionPool(host='ark.cn-beijing.volces.com', port=443): Read timed out. (read timeout=10)


处理进度:   2%|▏         | 4/222 [00:31<27:12,  7.49s/对]

API请求错误: HTTPSConnectionPool(host='ark.cn-beijing.volces.com', port=443): Read timed out. (read timeout=10)


...

API请求错误: HTTPSConnectionPool(host='ark.cn-beijing.volces.com', port=443): Read timed out. (read timeout=10)


处理进度: 100%|█████████▉| 221/222 [30:38<00:10, 10.92s/对]

API请求错误: HTTPSConnectionPool(host='ark.cn-beijing.volces.com', port=443): Read timed out. (read timeout=10)


处理进度: 100%|██████████| 222/222 [30:50<00:00,  8.33s/对]


分级统计结果:
简单: 130个
中等: 23个
困难: 2个
未知: 67个

所有问答难度分级已完成，结果保存在： D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA_level（QA对）(2).json


再修改提示词，改用模型：deepseek，结果未知变多，效果更差了

In [ ]:
import json
import requests
import time
from tqdm import tqdm
from opencc import OpenCC
import jieba

API_KEY = "ef364c37-1e5a-4c34-8768-17bc5bafa152"  
API_URL = "https://ark.cn-beijing.volces.com/api/v3/chat/completions"
HEADERS = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}

cc = OpenCC('t2s')

def tokenize(text):
    """中文分词函数"""
    return list(jieba.cut(text))

def build_prompt(question, answer):
    return f"""
你是一位保险领域的资深专家，请根据以下问答对的复杂程度和详细程度，将其划分为"简单"、"中等"或"困难"三个等级。

### 划分标准：
1. 简单：
   - 问题特征：问题表述清晰直接，指向明确，无需额外解读，通常询问单一、具体的信息，如办理某项手续所需的表格、某个事件的直接后果等。
   - 答案特征：答案能直接从文本中找到，无需对信息进行整合、推理或补充额外知识，问题与答案的对应关系一目了然。
   - 示例：问题 “使用宏利信用卡支付保费有哪些限制条件？”，答案清晰列出几个条件，表述直接。

2. 中等：
   - 问题特征：问题可能涉及多个方面或步骤，需要对相关信息进行一定的梳理和整合才能理解问题核心，或者需要在多个条件下进行判断。
   - 答案特征：答案需要对文本中的信息进行整合、归纳或一定的理解，可能涉及多个条件或步骤的组合，但不需要深入的逻辑推理或额外的专业知识背景。
   - 示例：问题 “如果保单账户余额不足导致无法扣除月费，会有什么后果？”，答案需考虑宽限期等情况进行说明。

3. 困难：
   - 问题特征：问题具有一定的复杂性，可能涉及专业概念、复杂逻辑关系或多条件的综合判断，需要对保险业务有一定的理解才能准确把握问题意图。
   - 答案特征：答案需要进行复杂的逻辑推理、多条件判断或结合一定的保险知识背景，问题和答案之间的关系不直观，需要深入分析文本内容。
   - 示例：问题 “如果保险的次位受益人是未成年人，为什么不能由首位受益人担任其信托人？”，答案需解释背后的逻辑矛盾。

   
### 评估维度：
1. 信息获取的难易程度
2. 问题涉及概念的复杂程度
3. 答案的复杂度和长度
4. 问题场景的复杂程度

### 待评估QA对：
问题：{question}
回答：{answer}

### 输出要求：
请严格按以下JSON格式回应：
{{
    "difficulty": "简单|中等|困难",
    "reason": "分级理由说明"
}}
"""

def classify_qa_pair(question, answer):
    prompt = build_prompt(question, answer)
    data = {
        "model": "deepseek-v3-250324",  
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.3,
        "response_format": {"type": "json_object"}  
    }
    
    try:
        response = requests.post(API_URL, headers=HEADERS, json=data, timeout=10)
        response.raise_for_status()  
        
        result = response.json()
        
        if "choices" in result:
            content = result["choices"][0]["message"]["content"]
        elif "result" in result:  
            content = result["result"]
        else:
            content = json.dumps({"difficulty": "未知", "reason": "API响应格式不符"})
            
        try:
            json.loads(content)  
            return content
        except json.JSONDecodeError:
            return json.dumps({"difficulty": "未知", "reason": "API返回非JSON格式"})
            
    except requests.exceptions.RequestException as e:
        print(f"API请求错误: {str(e)}")
        return json.dumps({"difficulty": "未知", "reason": f"API请求错误: {str(e)}"})
    except Exception as e:
        print(f"处理异常: {str(e)}")
        return json.dumps({"difficulty": "未知", "reason": f"处理异常: {str(e)}"})

def process_testset(qa_file, output_file):
    with open(qa_file, "r", encoding="utf-8") as f:
        qa_list = json.load(f)

    results = []
    stats = {"简单": 0, "中等": 0, "困难": 0, "未知": 0}
    
    print("\n开始处理QA对难度分级...\n")
    
    for item in tqdm(qa_list, desc="处理进度", unit="对"):
        question = item.get("question", "")
        answer = item.get("answer", "")
        
        try:
            classification = classify_qa_pair(question, answer)
            classification_data = json.loads(classification)
            
            item["difficulty"] = classification_data.get("difficulty", "未知")
            item["difficulty_reason"] = classification_data.get("reason", "")
            
            stats[item["difficulty"]] += 1
        except json.JSONDecodeError:
            item["difficulty"] = "未知"
            item["difficulty_reason"] = "响应解析失败"
            stats["未知"] += 1
        
        results.append(item)
        time.sleep(1)  

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    
    print("\n分级统计结果:")
    for level, count in stats.items():
        print(f"{level}: {count}个")
    print("\n所有问答难度分级已完成，结果保存在：", output_file)

if __name__ == "__main__":
    process_testset(
        r"D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA4.json",    
        r"D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA_level（QA对）(3).json"          
    )


开始处理QA对难度分级...



处理进度: 100%|██████████| 222/222 [17:46<00:00,  4.80s/对]


分级统计结果:
简单: 2个
中等: 37个
困难: 3个
未知: 180个

所有问答难度分级已完成，结果保存在： D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA_level（QA对）(3).json


放弃对已有QA对进行分等级，尝试结合生成QA对这一步，对提示词增加问题类型与难度等级，问题类型包括是什么、为什么、怎么做，难度等级为简单、中等、困难，模型：deepseek-r1
效果：问题类型都是是什么，难度等级简单较多

In [15]:
import os
import json
import re
from openai import OpenAI


class CHAT_MODEL:
    def __init__(self, api_key, base_url, model_name):
        self.llm = OpenAI(api_key=api_key, base_url=base_url)
        self.model_name = model_name

    def chat(self, user_prompt):
        completion = self.llm.chat.completions.create(
            model=self.model_name,
            messages=[{"role": "user", "content": user_prompt}],
        )
        return completion.choices[0].message.content


def extract_segments_with_page(md_text, max_segments=5, min_len=50):
    # 使用页码标题分割，如 ## 第3页
    page_splits = re.split(r'#+\s*第\s*(\d+)\s*页', md_text)
    segments = []

    if len(page_splits) <= 1:
        # 如果没有页码标识，按整段分
        raw_paragraphs = re.split(r'\n\s*\n', md_text)
        for para in raw_paragraphs:
            para = para.strip()
            if len(para) >= min_len:
                segments.append(("未知页码", para))
    else:
        for i in range(1, len(page_splits), 2):
            page = int(page_splits[i])
            # 跳过第 0 页和第 1 页
            if page in [0, 1,2]:
                continue
            content = page_splits[i + 1]
            paragraphs = re.split(r'\n\s*\n', content)
            for para in paragraphs:
                para = para.strip()
                if len(para) >= min_len:
                    segments.append((f"{page}", para))

    total = len(segments)
    if total <= max_segments:
        return segments
    step = total // max_segments
    return [segments[i] for i in range(0, total, step)][:max_segments]


def build_prompt(few_shot_examples, text_segment):
    prompt = (
        "作为资深保险条款分析师，请根据以下文档内容生成高质量问答对。要求如下：\n\n"
        "### 核心要求（保留您原有标准）\n"
        "1. 问题必须模拟真实业务场景需求，例如：\n"
        "   - 理赔流程、免责条款、适用场景、利益比较等\n"
        "   - 避免简单信息复述，要体现实际业务疑问\n"
        "2. 答案需满足：\n"
        "   - 基于文档内容但重组表达，不得直接照抄\n"
        "   - 逻辑严谨，信息完整准确\n"
        "   - 可跨段落综合，但需注明引用来源\n\n"
        "### 新增规范\n"
        "1. 问题类型标注（必须三选一）：\n"
        "   ▸ 【是什么】定义解释类（如：什么是保单现金价值？）\n"
        "   ▸ 【为什么】原因分析类（如：为什么需要健康告知？）\n"
        "   ▸ 【怎么做】操作流程类（如：如何办理保单贷款？）\n\n"
        "2. 难度等级标注（必须三选一）：\n"
        "   ▸ 【简单】单概念问题（答案1-2句话）\n"
        "   ▸ 【中等】多要素问题（答案含步骤或条件）\n"
        "   ▸ 【困难】综合分析问题（需跨章节引用）\n\n"
        "### 输出格式（升级版）\n"
        "```json\n"
        "{\n"
        '  "question": "问题内容（使用客户自然语言表达）",\n'
        '  "answer": "答案（分点陈述，必要时用序号标注步骤）",\n'
        '  "qa_type": "是什么/为什么/怎么做",\n'
        '  "difficulty": "简单/中等/困难",\n'
        '  "source_file": "文档名称",\n'
        '  "page_num": "页码",\n'
        '  "content": "引用的原始文本片段"\n'
        "}\n"
        "```\n\n"
        "### 您原有的问题范例（现增加分类标注）\n"
        "▸ 什么是...？ 【是什么】\n"
        "▸ 是否可以说...？ 【为什么】\n"
        "▸ 如果...会怎样？ 【怎么做】\n"
        "▸ 为什么...会发生？ 【为什么】\n"
        "▸ ...的好处是什么？ 【是什么】\n\n"
        "### 参考案例\n"
        "【示例1-理赔场景】\n" + few_shot_examples[0] + "\n\n"
        "【示例2-条款解释】\n" + few_shot_examples[1] + "\n\n"
        "### 待分析内容\n"
        f"{text_segment}\n\n"
        "请生成符合所有要求的JSON输出："
    )
    return prompt


def parse_json(response):
    lines = response.splitlines()
    start_index = None
    end_index = None
    for i, line in enumerate(lines):
        if '{' in line:
            start_index = i
        if '}' in line and start_index is not None:
            end_index = i
            break
    if start_index is not None and end_index is not None:
        json_str = '\n'.join(lines[start_index:end_index + 1])
        try:
            return json.loads(json_str)
        except json.JSONDecodeError as e:
            print(f"JSON 解析错误: {e}")
    return None


def main():
    input_folder = r"D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\data\(7)"
    output_file = r"D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA6(7)(2).json"
    qa_per_doc = 16
    api_key = "c751d97c-ba64-4a9a-afad-a0db5b88dc37"
    base_url = "https://ark.cn-beijing.volces.com/api/v3"
    model_name = "deepseek-r1-distill-qwen-32b-250120"

    chat_model = CHAT_MODEL(api_key=api_key, base_url=base_url, model_name=model_name)

    few_shot_examples = [
        '''{
"question": "投资相连寿险保单过了宽限期还没缴费会怎样？",
"answer": "灵活投资宝/万利保障计划/宏宝保单将享有30日宽限期。若过后仍未缴费，客户会收到保单失效通知书，说明如何复效。",
"source_file": "电子行政运作手册_保单行政_2023_10.md",
"page_num": "6",
"content": "投資相連壽險計劃保單...將享有30日的寬限期..." }''',
        '''{
"question": "什么情况下保险顾问的营业积分会被扣回？",
"answer": "若某月结日的净已缴保费低于两个月前月结日的净已缴保费，保险顾问的积分将被扣回。",
"source_file": "电子行政运作手册_保单行政_2023_10.md",
"page_num": "7",
"content": "於第一年淨保費測試...若（N）月結日的淨已繳保費 ≤（N–2）月結日..." }'''
    ]

    results = []

    for filename in os.listdir(input_folder):
        if not filename.endswith(".md"):
            continue
        file_path = os.path.join(input_folder, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()

        segments = extract_segments_with_page(content, max_segments=qa_per_doc)

        for idx, (page, segment) in enumerate(segments):
            prompt = build_prompt(few_shot_examples, segment)
            max_retries =2
            for retry in range(max_retries):
                try:
                    response = chat_model.chat(prompt)
                    parsed = parse_json(response)
                    if parsed is not None:
                        parsed["source_file"] = filename
                        parsed["page_num"] = page
                        parsed["content"] = segment
                        results.append(parsed)
                        break
                    else:
                        print(f"重试 {retry + 1}/{max_retries} - 文件 {filename} 第{idx + 1}段（页码：{page}）：无法解析 JSON")
                except Exception as e:
                    print(f"重试 {retry + 1}/{max_retries} - 文件 {filename} 第{idx + 1}段（页码：{page}）：{e}")
            else:
                print(f"错误 - 文件 {filename} 第{idx + 1}段（页码：{page}）：达到最大重试次数，无法获取有效数据")

    valid_results = []
    for result in results:
        question = result.get("question", "").strip()
        answer = result.get("answer", "").strip()
        if len(question) > 5 and len(answer) > 10:
            valid_results.append(result)

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(valid_results, f, ensure_ascii=False, indent=2)

    print(f"\n成功保存 {len(valid_results)} 个高质量 QA 对到 {output_file}")


if __name__ == "__main__":
    main()

JSON 解析错误: Invalid \escape: line 8 column 189 (char 544)
重试 1/2 - 文件 守护无间危疾保 保单条款_2022_07.md 第12段（页码：33）：无法解析 JSON
JSON 解析错误: Invalid \escape: line 8 column 192 (char 574)
重试 2/2 - 文件 守护无间危疾保 保单条款_2022_07.md 第12段（页码：33）：无法解析 JSON
错误 - 文件 守护无间危疾保 保单条款_2022_07.md 第12段（页码：33）：达到最大重试次数，无法获取有效数据
JSON 解析错误: Invalid \escape: line 8 column 203 (char 527)
重试 1/2 - 文件 守护无间危疾保 保单条款_2022_07.md 第15段（页码：39）：无法解析 JSON
JSON 解析错误: Invalid \escape: line 8 column 203 (char 567)
重试 2/2 - 文件 守护无间危疾保 保单条款_2022_07.md 第15段（页码：39）：无法解析 JSON
错误 - 文件 守护无间危疾保 保单条款_2022_07.md 第15段（页码：39）：达到最大重试次数，无法获取有效数据
JSON 解析错误: Invalid \escape: line 8 column 53 (char 293)
重试 1/2 - 文件 守护无间危疾保 保单条款_2022_07.md 第16段（页码：42）：无法解析 JSON
JSON 解析错误: Invalid \escape: line 8 column 53 (char 263)
重试 2/2 - 文件 守护无间危疾保 保单条款_2022_07.md 第16段（页码：42）：无法解析 JSON
错误 - 文件 守护无间危疾保 保单条款_2022_07.md 第16段（页码：42）：达到最大重试次数，无法获取有效数据

成功保存 13 个高质量 QA 对到 D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA6(7)(2).json


！！！！丰富提示词，模型：deepseek-r1，新增比例规定，3：5：2和3：3：4
效果：难度等级比例符合要求，但问题类型都是 是什么

In [17]:
import os
import json
import re
from openai import OpenAI


class CHAT_MODEL:
    def __init__(self, api_key, base_url, model_name):
        self.llm = OpenAI(api_key=api_key, base_url=base_url)
        self.model_name = model_name

    def chat(self, user_prompt):
        completion = self.llm.chat.completions.create(
            model=self.model_name,
            messages=[{"role": "user", "content": user_prompt}],
        )
        return completion.choices[0].message.content


def extract_segments_with_page(md_text, max_segments=5, min_len=50):
    # 使用页码标题分割，如 ## 第3页
    page_splits = re.split(r'#+\s*第\s*(\d+)\s*页', md_text)
    segments = []

    if len(page_splits) <= 1:
        # 如果没有页码标识，按整段分
        raw_paragraphs = re.split(r'\n\s*\n', md_text)
        for para in raw_paragraphs:
            para = para.strip()
            if len(para) >= min_len:
                segments.append(("未知页码", para))
    else:
        for i in range(1, len(page_splits), 2):
            page = int(page_splits[i])
            # 跳过第 0 页和第 1 页
            if page in [0, 1,2]:
                continue
            content = page_splits[i + 1]
            paragraphs = re.split(r'\n\s*\n', content)
            for para in paragraphs:
                para = para.strip()
                if len(para) >= min_len:
                    segments.append((f"{page}", para))

    total = len(segments)
    if total <= max_segments:
        return segments
    step = total // max_segments
    return [segments[i] for i in range(0, total, step)][:max_segments]


def build_prompt(few_shot_examples, text_segment):
    prompt = (
        "你是一个资深保险条款分析师，任务是基于以下多个保险文档内容，生成用户可能会提出的业务相关问题，并结合文档信息提供专业、准确、结构清晰的回答。\n\n"
        "请根据以下文档内容，生成一个高质量的问题和答案对。问题应贴近实际业务场景，答案应清晰、信息丰富。\n\n"
        "### 核心要求（保留您原有标准）\n"
        "1. 问题必须模拟真实用户的业务需求或疑问，例如理赔流程、免责条款、适用场景、利益比较等，不是简单的信息回述或定义解释。\n"
        "2. 答案可以综合多个文档片段的内容，但必须确保准确，逻辑严谨，不得凭空捏造。\n"
        "3. 回答中如引用到不同文档，请明确列出引用的文档名称与页码。\n\n"
        "4. 问题属于三种难度等级的数量要尽量符合简单：中等：困难=3：5：2。\n"
        "问题部分：围绕一个核心主题提出一个问题，语言表达应贴近用户常见问法，如：\n"
        "    - 什么是...？\n"
        "    - 是否可以说...？\n"
        "    - 请解释一下...的含义\n"
        "    - 如果...会怎样？\n"
        "    - 能否举个例子说明...？\n"
        "    - 为什么...会发生？\n"
        "    - ...的好处是什么？\n"
        "答案部分：逻辑清晰，内容完整。不得照抄原文，应体现理解和重组能力。\n\n"
        "### 新增规范\n"
        "1. 问题类型标注（必须三选一）：\n"
        "   ▸ 【是什么】定义解释类（如：什么是保单现金价值？）\n"
        "   ▸ 【为什么】原因分析类（如：为什么需要健康告知？）\n"
        "   ▸ 【怎么做】操作流程类（如：如何办理保单贷款？）\n\n"
        "2. 难度等级标注（必须三选一）：\n"
        "   ▸ 【简单】单概念问题（答案1-2句话）\n"
        "   ▸ 【中等】多要素问题（答案含步骤或条件）\n"
        "   ▸ 【困难】综合分析问题（需跨章节引用）\n\n"
        "### 输出格式（升级版）\n"
        "```json\n"
        "{\n"
        '  "question": "问题内容（使用客户自然语言表达）",\n'
        '  "answer": "答案内容",\n'
        '  "qa_type": "是什么/为什么/怎么做",\n'
        '  "difficulty": "简单/中等/困难",\n'
        '  "source_file": "文档名称",\n'
        '  "page_num": "页码",\n'
        '  "content": "引用的原始文本片段"\n'
        "}\n"
        "```\n\n"
        "### 参考案例\n"
        "【示例1-理赔场景】\n" + few_shot_examples[0] + "\n\n"
        "【示例2-条款解释】\n" + few_shot_examples[1] + "\n\n"
        "### 待分析内容\n"
        f"{text_segment}\n\n"
        "请生成符合所有要求的JSON输出："
    )
    return prompt


def parse_json(response):
    lines = response.splitlines()
    start_index = None
    end_index = None
    for i, line in enumerate(lines):
        if '{' in line:
            start_index = i
        if '}' in line and start_index is not None:
            end_index = i
            break
    if start_index is not None and end_index is not None:
        json_str = '\n'.join(lines[start_index:end_index + 1])
        try:
            return json.loads(json_str)
        except json.JSONDecodeError as e:
            print(f"JSON 解析错误: {e}")
    return None


def main():
    input_folder = r"D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\data\(7)"
    output_file = r"D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA6(7)(352).json"
    qa_per_doc = 10
    api_key = "c751d97c-ba64-4a9a-afad-a0db5b88dc37"
    base_url = "https://ark.cn-beijing.volces.com/api/v3"
    model_name = "deepseek-r1-distill-qwen-32b-250120"

    chat_model = CHAT_MODEL(api_key=api_key, base_url=base_url, model_name=model_name)

    few_shot_examples = [
        '''{
"question": "投资相连寿险保单过了宽限期还没缴费会怎样？",
"answer": "灵活投资宝/万利保障计划/宏宝保单将享有30日宽限期。若过后仍未缴费，客户会收到保单失效通知书，说明如何复效。",
"source_file": "电子行政运作手册_保单行政_2023_10.md",
"page_num": "6",
"content": "投資相連壽險計劃保單...將享有30日的寬限期..." }''',
        '''{
"question": "什么情况下保险顾问的营业积分会被扣回？",
"answer": "若某月结日的净已缴保费低于两个月前月结日的净已缴保费，保险顾问的积分将被扣回。",
"source_file": "电子行政运作手册_保单行政_2023_10.md",
"page_num": "7",
"content": "於第一年淨保費測試...若（N）月結日的淨已繳保費 ≤（N–2）月結日..." }'''
    ]

    results = []

    for filename in os.listdir(input_folder):
        if not filename.endswith(".md"):
            continue
        file_path = os.path.join(input_folder, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()

        segments = extract_segments_with_page(content, max_segments=qa_per_doc)

        for idx, (page, segment) in enumerate(segments):
            prompt = build_prompt(few_shot_examples, segment)
            max_retries =2
            for retry in range(max_retries):
                try:
                    response = chat_model.chat(prompt)
                    parsed = parse_json(response)
                    if parsed is not None:
                        parsed["source_file"] = filename
                        parsed["page_num"] = page
                        parsed["content"] = segment
                        results.append(parsed)
                        break
                    else:
                        print(f"重试 {retry + 1}/{max_retries} - 文件 {filename} 第{idx + 1}段（页码：{page}）：无法解析 JSON")
                except Exception as e:
                    print(f"重试 {retry + 1}/{max_retries} - 文件 {filename} 第{idx + 1}段（页码：{page}）：{e}")
            else:
                print(f"错误 - 文件 {filename} 第{idx + 1}段（页码：{page}）：达到最大重试次数，无法获取有效数据")

    valid_results = []
    for result in results:
        question = result.get("question", "").strip()
        answer = result.get("answer", "").strip()
        if len(question) > 5 and len(answer) > 10:
            valid_results.append(result)

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(valid_results, f, ensure_ascii=False, indent=2)

    print(f"\n成功保存 {len(valid_results)} 个高质量 QA 对到 {output_file}")


if __name__ == "__main__":
    main()

JSON 解析错误: Invalid \escape: line 8 column 70 (char 283)
重试 1/2 - 文件 守护无间危疾保 保单条款_2022_07.md 第7段（页码：31）：无法解析 JSON
JSON 解析错误: Invalid \escape: line 8 column 70 (char 284)
重试 2/2 - 文件 守护无间危疾保 保单条款_2022_07.md 第7段（页码：31）：无法解析 JSON
错误 - 文件 守护无间危疾保 保单条款_2022_07.md 第7段（页码：31）：达到最大重试次数，无法获取有效数据

成功保存 9 个高质量 QA 对到 D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA6(7)(352).json


In [20]:
import os
import json
import re
from openai import OpenAI


class CHAT_MODEL:
    def __init__(self, api_key, base_url, model_name):
        self.llm = OpenAI(api_key=api_key, base_url=base_url)
        self.model_name = model_name

    def chat(self, user_prompt):
        completion = self.llm.chat.completions.create(
            model=self.model_name,
            messages=[{"role": "user", "content": user_prompt}],
        )
        return completion.choices[0].message.content


def extract_segments_with_page(md_text, max_segments=5, min_len=50):
    # 使用页码标题分割，如 ## 第3页
    page_splits = re.split(r'#+\s*第\s*(\d+)\s*页', md_text)
    segments = []

    if len(page_splits) <= 1:
        # 如果没有页码标识，按整段分
        raw_paragraphs = re.split(r'\n\s*\n', md_text)
        for para in raw_paragraphs:
            para = para.strip()
            if len(para) >= min_len:
                segments.append(("未知页码", para))
    else:
        for i in range(1, len(page_splits), 2):
            page = int(page_splits[i])
            # 跳过第 0 页和第 1 页
            if page in [0, 1,2]:
                continue
            content = page_splits[i + 1]
            paragraphs = re.split(r'\n\s*\n', content)
            for para in paragraphs:
                para = para.strip()
                if len(para) >= min_len:
                    segments.append((f"{page}", para))

    total = len(segments)
    if total <= max_segments:
        return segments
    step = total // max_segments
    return [segments[i] for i in range(0, total, step)][:max_segments]


def build_prompt(few_shot_examples, text_segment):
    prompt = (
        "你是一个资深保险条款分析师，任务是基于以下多个保险文档内容，生成用户可能会提出的业务相关问题，并结合文档信息提供专业、准确、结构清晰的回答。\n\n"
        "请根据以下文档内容，生成一个高质量的问题和答案对。问题应贴近实际业务场景，答案应清晰、信息丰富。\n\n"
        "### 核心要求（保留您原有标准）\n"
        "1. 问题必须模拟真实用户的业务需求或疑问，例如理赔流程、免责条款、适用场景、利益比较等，不是简单的信息回述或定义解释。\n"
        "2. 答案可以综合多个文档片段的内容，但必须确保准确，逻辑严谨，不得凭空捏造。\n"
        "3. 回答中如引用到不同文档，请明确列出引用的文档名称与页码。\n\n"
        "4. 问题属于三种难度等级的数量要尽量符合简单：中等：困难=3：5：2，问题的三种类型的数量要尽量符合是什么：为什么：怎么做=3：3：4。\n"
        "问题部分：围绕一个核心主题提出一个问题，语言表达应贴近用户常见问法，如：\n"
        "    - 什么是...？\n"
        "    - 是否可以说...？\n"
        "    - 请解释一下...的含义\n"
        "    - 如果...会怎样？\n"
        "    - 能否举个例子说明...？\n"
        "    - 为什么...会发生？\n"
        "    - ...的好处是什么？\n"
        "答案部分：逻辑清晰，内容完整。不得照抄原文，应体现理解和重组能力。\n\n"
        "### 新增规范\n"
        "1. 问题类型标注：\n"
        "   ▸ 【是什么】定义解释类（如：什么是保单现金价值？）\n"
        "   ▸ 【为什么】原因分析类（如：为什么需要健康告知？）\n"
        "   ▸ 【怎么做】操作流程类（如：如何办理保单贷款？）\n\n"
        "2. 难度等级标注：\n"
        "   ▸ 【简单】单概念问题（答案1-2句话）\n"
        "   ▸ 【中等】多要素问题（答案含步骤或条件）\n"
        "   ▸ 【困难】综合分析问题（需跨章节引用）\n\n"
        "### 输出格式（升级版）\n"
        "```json\n"
        "{\n"
        '  "question": "问题内容（使用客户自然语言表达）",\n'
        '  "answer": "答案内容",\n'
        '  "qa_type": "是什么/为什么/怎么做",\n'
        '  "difficulty": "简单/中等/困难",\n'
        '  "source_file": "文档名称",\n'
        '  "page_num": "页码",\n'
        '  "content": "引用的原始文本片段"\n'
        "}\n"
        "```\n\n"
        "### 参考案例\n"
        "【示例1-理赔场景】\n" + few_shot_examples[0] + "\n\n"
        "【示例2-条款解释】\n" + few_shot_examples[1] + "\n\n"
        "### 待分析内容\n"
        f"{text_segment}\n\n"
        "请生成符合所有要求的JSON输出："
    )
    return prompt


def parse_json(response):
    lines = response.splitlines()
    start_index = None
    end_index = None
    for i, line in enumerate(lines):
        if '{' in line:
            start_index = i
        if '}' in line and start_index is not None:
            end_index = i
            break
    if start_index is not None and end_index is not None:
        json_str = '\n'.join(lines[start_index:end_index + 1])
        try:
            return json.loads(json_str)
        except json.JSONDecodeError as e:
            print(f"JSON 解析错误: {e}")
    return None


def main():
    input_folder = r"D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\data\(7)"
    output_file = r"D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA6(7)(352)(334).json"
    qa_per_doc = 10
    api_key = "c751d97c-ba64-4a9a-afad-a0db5b88dc37"
    base_url = "https://ark.cn-beijing.volces.com/api/v3"
    model_name = "deepseek-r1-distill-qwen-32b-250120"

    chat_model = CHAT_MODEL(api_key=api_key, base_url=base_url, model_name=model_name)

    few_shot_examples = [
        '''{
"question": "投资相连寿险保单过了宽限期还没缴费会怎样？",
"answer": "灵活投资宝/万利保障计划/宏宝保单将享有30日宽限期。若过后仍未缴费，客户会收到保单失效通知书，说明如何复效。",
"source_file": "电子行政运作手册_保单行政_2023_10.md",
"page_num": "6",
"content": "投資相連壽險計劃保單...將享有30日的寬限期..." }''',
        '''{
"question": "什么情况下保险顾问的营业积分会被扣回？",
"answer": "若某月结日的净已缴保费低于两个月前月结日的净已缴保费，保险顾问的积分将被扣回。",
"source_file": "电子行政运作手册_保单行政_2023_10.md",
"page_num": "7",
"content": "於第一年淨保費測試...若（N）月結日的淨已繳保費 ≤（N–2）月結日..." }'''
    ]

    results = []

    for filename in os.listdir(input_folder):
        if not filename.endswith(".md"):
            continue
        file_path = os.path.join(input_folder, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()

        segments = extract_segments_with_page(content, max_segments=qa_per_doc)

        for idx, (page, segment) in enumerate(segments):
            prompt = build_prompt(few_shot_examples, segment)
            max_retries =2
            for retry in range(max_retries):
                try:
                    response = chat_model.chat(prompt)
                    parsed = parse_json(response)
                    if parsed is not None:
                        parsed["source_file"] = filename
                        parsed["page_num"] = page
                        parsed["content"] = segment
                        results.append(parsed)
                        break
                    else:
                        print(f"重试 {retry + 1}/{max_retries} - 文件 {filename} 第{idx + 1}段（页码：{page}）：无法解析 JSON")
                except Exception as e:
                    print(f"重试 {retry + 1}/{max_retries} - 文件 {filename} 第{idx + 1}段（页码：{page}）：{e}")
            else:
                print(f"错误 - 文件 {filename} 第{idx + 1}段（页码：{page}）：达到最大重试次数，无法获取有效数据")

    valid_results = []
    for result in results:
        question = result.get("question", "").strip()
        answer = result.get("answer", "").strip()
        if len(question) > 5 and len(answer) > 10:
            valid_results.append(result)

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(valid_results, f, ensure_ascii=False, indent=2)

    print(f"\n成功保存 {len(valid_results)} 个高质量 QA 对到 {output_file}")


if __name__ == "__main__":
    main()

JSON 解析错误: Invalid \escape: line 8 column 70 (char 275)
重试 1/2 - 文件 守护无间危疾保 保单条款_2022_07.md 第7段（页码：31）：无法解析 JSON
JSON 解析错误: Invalid \escape: line 8 column 70 (char 272)
重试 2/2 - 文件 守护无间危疾保 保单条款_2022_07.md 第7段（页码：31）：无法解析 JSON
错误 - 文件 守护无间危疾保 保单条款_2022_07.md 第7段（页码：31）：达到最大重试次数，无法获取有效数据

成功保存 9 个高质量 QA 对到 D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA6(7)(352)(334).json


！！！因为类型不成比例，换模型试试（豆包），效果：难度等级未出现困难等级，问题类型出现了两个怎么做，但依旧是什么最多

In [21]:
import os
import json
import re
from openai import OpenAI


class CHAT_MODEL:
    def __init__(self, api_key, base_url, model_name):
        self.llm = OpenAI(api_key=api_key, base_url=base_url)
        self.model_name = model_name

    def chat(self, user_prompt):
        completion = self.llm.chat.completions.create(
            model=self.model_name,
            messages=[{"role": "user", "content": user_prompt}],
        )
        return completion.choices[0].message.content


def extract_segments_with_page(md_text, max_segments=5, min_len=50):
    # 使用页码标题分割，如 ## 第3页
    page_splits = re.split(r'#+\s*第\s*(\d+)\s*页', md_text)
    segments = []

    if len(page_splits) <= 1:
        # 如果没有页码标识，按整段分
        raw_paragraphs = re.split(r'\n\s*\n', md_text)
        for para in raw_paragraphs:
            para = para.strip()
            if len(para) >= min_len:
                segments.append(("未知页码", para))
    else:
        for i in range(1, len(page_splits), 2):
            page = int(page_splits[i])
            # 跳过第 0 页和第 1 页
            if page in [0, 1,2]:
                continue
            content = page_splits[i + 1]
            paragraphs = re.split(r'\n\s*\n', content)
            for para in paragraphs:
                para = para.strip()
                if len(para) >= min_len:
                    segments.append((f"{page}", para))

    total = len(segments)
    if total <= max_segments:
        return segments
    step = total // max_segments
    return [segments[i] for i in range(0, total, step)][:max_segments]


def build_prompt(few_shot_examples, text_segment):
    prompt = (
        "你是一个资深保险条款分析师，任务是基于以下多个保险文档内容，生成用户可能会提出的业务相关问题，并结合文档信息提供专业、准确、结构清晰的回答。\n\n"
        "请根据以下文档内容，生成一个高质量的问题和答案对。问题应贴近实际业务场景，答案应清晰、信息丰富。\n\n"
        "### 核心要求（保留您原有标准）\n"
        "1. 问题必须模拟真实用户的业务需求或疑问，例如理赔流程、免责条款、适用场景、利益比较等，不是简单的信息回述或定义解释。\n"
        "2. 答案可以综合多个文档片段的内容，但必须确保准确，逻辑严谨，不得凭空捏造。\n"
        "3. 回答中如引用到不同文档，请明确列出引用的文档名称与页码。\n\n"
        "4. 问题属于三种难度等级的数量要尽量符合简单：中等：困难=3：5：2，问题的三种类型的数量要尽量符合是什么：为什么：怎么做=3：3：4。\n"
        "问题部分：围绕一个核心主题提出一个问题，语言表达应贴近用户常见问法，如：\n"
        "    - 什么是...？\n"
        "    - 是否可以说...？\n"
        "    - 请解释一下...的含义\n"
        "    - 如果...会怎样？\n"
        "    - 能否举个例子说明...？\n"
        "    - 为什么...会发生？\n"
        "    - ...的好处是什么？\n"
        "答案部分：逻辑清晰，内容完整。不得照抄原文，应体现理解和重组能力。\n\n"
        "### 新增规范\n"
        "1. 问题类型标注：\n"
        "   ▸ 【是什么】定义解释类（如：什么是保单现金价值？）\n"
        "   ▸ 【为什么】原因分析类（如：为什么需要健康告知？）\n"
        "   ▸ 【怎么做】操作流程类（如：如何办理保单贷款？）\n\n"
        "2. 难度等级标注：\n"
        "   ▸ 【简单】单概念问题（答案1-2句话）\n"
        "   ▸ 【中等】多要素问题（答案含步骤或条件）\n"
        "   ▸ 【困难】综合分析问题（需跨章节引用）\n\n"
        "### 输出格式（升级版）\n"
        "```json\n"
        "{\n"
        '  "question": "问题内容（使用客户自然语言表达）",\n'
        '  "answer": "答案内容",\n'
        '  "qa_type": "是什么/为什么/怎么做",\n'
        '  "difficulty": "简单/中等/困难",\n'
        '  "source_file": "文档名称",\n'
        '  "page_num": "页码",\n'
        '  "content": "引用的原始文本片段"\n'
        "}\n"
        "```\n\n"
        "### 参考案例\n"
        "【示例1-理赔场景】\n" + few_shot_examples[0] + "\n\n"
        "【示例2-条款解释】\n" + few_shot_examples[1] + "\n\n"
        "### 待分析内容\n"
        f"{text_segment}\n\n"
        "请生成符合所有要求的JSON输出："
    )
    return prompt


def parse_json(response):
    lines = response.splitlines()
    start_index = None
    end_index = None
    for i, line in enumerate(lines):
        if '{' in line:
            start_index = i
        if '}' in line and start_index is not None:
            end_index = i
            break
    if start_index is not None and end_index is not None:
        json_str = '\n'.join(lines[start_index:end_index + 1])
        try:
            return json.loads(json_str)
        except json.JSONDecodeError as e:
            print(f"JSON 解析错误: {e}")
    return None


def main():
    input_folder = r"D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\data\(7)"
    output_file = r"D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA6(7)(352)(334)豆包.json"
    qa_per_doc = 10
    api_key = "c751d97c-ba64-4a9a-afad-a0db5b88dc37"
    base_url = "https://ark.cn-beijing.volces.com/api/v3"
    model_name = "doubao-1-5-pro-32k-250115"

    chat_model = CHAT_MODEL(api_key=api_key, base_url=base_url, model_name=model_name)

    few_shot_examples = [
        '''{
"question": "投资相连寿险保单过了宽限期还没缴费会怎样？",
"answer": "灵活投资宝/万利保障计划/宏宝保单将享有30日宽限期。若过后仍未缴费，客户会收到保单失效通知书，说明如何复效。",
"source_file": "电子行政运作手册_保单行政_2023_10.md",
"page_num": "6",
"content": "投資相連壽險計劃保單...將享有30日的寬限期..." }''',
        '''{
"question": "什么情况下保险顾问的营业积分会被扣回？",
"answer": "若某月结日的净已缴保费低于两个月前月结日的净已缴保费，保险顾问的积分将被扣回。",
"source_file": "电子行政运作手册_保单行政_2023_10.md",
"page_num": "7",
"content": "於第一年淨保費測試...若（N）月結日的淨已繳保費 ≤（N–2）月結日..." }'''
    ]

    results = []

    for filename in os.listdir(input_folder):
        if not filename.endswith(".md"):
            continue
        file_path = os.path.join(input_folder, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()

        segments = extract_segments_with_page(content, max_segments=qa_per_doc)

        for idx, (page, segment) in enumerate(segments):
            prompt = build_prompt(few_shot_examples, segment)
            max_retries =2
            for retry in range(max_retries):
                try:
                    response = chat_model.chat(prompt)
                    parsed = parse_json(response)
                    if parsed is not None:
                        parsed["source_file"] = filename
                        parsed["page_num"] = page
                        parsed["content"] = segment
                        results.append(parsed)
                        break
                    else:
                        print(f"重试 {retry + 1}/{max_retries} - 文件 {filename} 第{idx + 1}段（页码：{page}）：无法解析 JSON")
                except Exception as e:
                    print(f"重试 {retry + 1}/{max_retries} - 文件 {filename} 第{idx + 1}段（页码：{page}）：{e}")
            else:
                print(f"错误 - 文件 {filename} 第{idx + 1}段（页码：{page}）：达到最大重试次数，无法获取有效数据")

    valid_results = []
    for result in results:
        question = result.get("question", "").strip()
        answer = result.get("answer", "").strip()
        if len(question) > 5 and len(answer) > 10:
            valid_results.append(result)

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(valid_results, f, ensure_ascii=False, indent=2)

    print(f"\n成功保存 {len(valid_results)} 个高质量 QA 对到 {output_file}")


if __name__ == "__main__":
    main()

JSON 解析错误: Invalid \escape: line 8 column 64 (char 294)
重试 1/2 - 文件 守护无间危疾保 保单条款_2022_07.md 第3段（页码：11）：无法解析 JSON
JSON 解析错误: Invalid \escape: line 8 column 64 (char 302)
重试 2/2 - 文件 守护无间危疾保 保单条款_2022_07.md 第3段（页码：11）：无法解析 JSON
错误 - 文件 守护无间危疾保 保单条款_2022_07.md 第3段（页码：11）：达到最大重试次数，无法获取有效数据
JSON 解析错误: Invalid \escape: line 8 column 70 (char 274)
重试 1/2 - 文件 守护无间危疾保 保单条款_2022_07.md 第7段（页码：31）：无法解析 JSON
JSON 解析错误: Invalid \escape: line 8 column 70 (char 280)
重试 2/2 - 文件 守护无间危疾保 保单条款_2022_07.md 第7段（页码：31）：无法解析 JSON
错误 - 文件 守护无间危疾保 保单条款_2022_07.md 第7段（页码：31）：达到最大重试次数，无法获取有效数据

成功保存 8 个高质量 QA 对到 D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA6(7)(352)(334)豆包.json


继续改提示词，为了增加问题类型，效果比改之前还不好，换了个内容多一点的文档跑，结果出现了两个怎么做类型，于是继续增多QA对数量（看是否是数量太少的原因），结果怎么做和是什么的比例还是1：4，可能是保单性质导致，同时难度等级未出现困难

In [26]:
import os
import json
import re
from openai import OpenAI


class CHAT_MODEL:
    def __init__(self, api_key, base_url, model_name):
        self.llm = OpenAI(api_key=api_key, base_url=base_url)
        self.model_name = model_name

    def chat(self, user_prompt):
        completion = self.llm.chat.completions.create(
            model=self.model_name,
            messages=[{"role": "user", "content": user_prompt}],
        )
        return completion.choices[0].message.content


def extract_segments_with_page(md_text, max_segments=5, min_len=50):
    # 使用页码标题分割，如 ## 第3页
    page_splits = re.split(r'#+\s*第\s*(\d+)\s*页', md_text)
    segments = []

    if len(page_splits) <= 1:
        # 如果没有页码标识，按整段分
        raw_paragraphs = re.split(r'\n\s*\n', md_text)
        for para in raw_paragraphs:
            para = para.strip()
            if len(para) >= min_len:
                segments.append(("未知页码", para))
    else:
        for i in range(1, len(page_splits), 2):
            page = int(page_splits[i])
            # 跳过第 0 页和第 1 页
            if page in [0, 1,2]:
                continue
            content = page_splits[i + 1]
            paragraphs = re.split(r'\n\s*\n', content)
            for para in paragraphs:
                para = para.strip()
                if len(para) >= min_len:
                    segments.append((f"{page}", para))

    total = len(segments)
    if total <= max_segments:
        return segments
    step = total // max_segments
    return [segments[i] for i in range(0, total, step)][:max_segments]


def build_prompt(few_shot_examples, text_segment):
    prompt = (
        "你是一个资深保险条款分析师，任务是基于以下多个保险文档内容，生成用户可能会提出的业务相关问题，并结合文档信息提供专业、准确、结构清晰的回答。\n\n"
        "请根据以下文档内容，生成一个高质量的问题和答案对。问题应贴近实际业务场景，答案应清晰、信息丰富。\n\n"
        "### 核心要求（保留您原有标准）\n"
        "1. 问题必须模拟真实用户的业务需求或疑问，例如理赔流程、免责条款、适用场景、利益比较等，不是简单的信息回述或定义解释。\n"
        "2. 答案可以综合多个文档片段的内容，但必须确保准确，逻辑严谨，不得凭空捏造。\n"
        "3. 回答中如引用到不同文档，请明确列出引用的文档名称与页码。\n\n"
        "4. 三种难度等级的问题的数量比例要符合简单：中等：困难=3：5：2。\n"
        "5. 三种类型问题的数量比例必须符合是什么：为什么：怎么做=3：3：4。\n"
        "问题部分：围绕一个核心主题提出一个问题，语言表达应贴近用户常见问法，如：\n"
        "    - 什么是...？\n"
        "    - 是否可以说...？\n"
        "    - 请解释一下...的含义\n"
        "    - 如果...会怎样？\n"
        "    - 如果...要怎么做？\n"
        "    - 如何...？\n"
        "    - 为什么...会发生？\n"
        "    - 为什么要...？\n"
        "    - ...的好处是什么？\n"
        "答案部分：逻辑清晰，内容完整。不得照抄原文，应体现理解和重组能力。\n\n"
        "### 新增规范\n"
        "1. 问题类型标注：\n"
        "   ▸ 【是什么】定义解释类（如：什么是保单现金价值？）\n"
        "   ▸ 【为什么】原因分析类（如：为什么需要健康告知？）\n"
        "   ▸ 【怎么做】操作流程类（如：如何办理保单贷款？）\n\n"
        "2. 难度等级标注：\n"
        "   ▸ 【简单】单概念问题\n"
        "   ▸ 【中等】多要素问题\n"
        "   ▸ 【困难】综合分析问题\n\n"
        "### 输出格式（升级版）\n"
        "```json\n"
        "{\n"
        '  "question": "问题内容（使用客户自然语言表达）",\n'
        '  "answer": "答案内容",\n'
        '  "qa_type": "是什么/为什么/怎么做",\n'
        '  "difficulty": "简单/中等/困难",\n'
        '  "source_file": "文档名称",\n'
        '  "page_num": "页码",\n'
        '  "content": "引用的原始文本片段"\n'
        "}\n"
        "```\n\n"
        "### 参考案例\n"
        "【示例1-理赔场景】\n" + few_shot_examples[0] + "\n\n"
        "【示例2-条款解释】\n" + few_shot_examples[1] + "\n\n"
        "### 待分析内容\n"
        f"{text_segment}\n\n"
        "请生成符合所有要求的JSON输出："
    )
    return prompt


def parse_json(response):
    lines = response.splitlines()
    start_index = None
    end_index = None
    for i, line in enumerate(lines):
        if '{' in line:
            start_index = i
        if '}' in line and start_index is not None:
            end_index = i
            break
    if start_index is not None and end_index is not None:
        json_str = '\n'.join(lines[start_index:end_index + 1])
        try:
            return json.loads(json_str)
        except json.JSONDecodeError as e:
            print(f"JSON 解析错误: {e}")
    return None


def main():
    input_folder = r"D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\data\(4)"
    output_file = r"D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA6(4)(352)(334)豆包2.json"
    qa_per_doc = 30
    api_key = "c751d97c-ba64-4a9a-afad-a0db5b88dc37"
    base_url = "https://ark.cn-beijing.volces.com/api/v3"
    model_name = "doubao-1-5-pro-32k-250115"

    chat_model = CHAT_MODEL(api_key=api_key, base_url=base_url, model_name=model_name)

    few_shot_examples = [
        '''{
"question": "投资相连寿险保单过了宽限期还没缴费会怎样？",
"answer": "灵活投资宝/万利保障计划/宏宝保单将享有30日宽限期。若过后仍未缴费，客户会收到保单失效通知书，说明如何复效。",
"source_file": "电子行政运作手册_保单行政_2023_10.md",
"page_num": "6",
"content": "投資相連壽險計劃保單...將享有30日的寬限期..." }''',
        '''{
"question": "什么情况下保险顾问的营业积分会被扣回？",
"answer": "若某月结日的净已缴保费低于两个月前月结日的净已缴保费，保险顾问的积分将被扣回。",
"source_file": "电子行政运作手册_保单行政_2023_10.md",
"page_num": "7",
"content": "於第一年淨保費測試...若（N）月結日的淨已繳保費 ≤（N–2）月結日..." }'''
    ]

    results = []

    for filename in os.listdir(input_folder):
        if not filename.endswith(".md"):
            continue
        file_path = os.path.join(input_folder, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()

        segments = extract_segments_with_page(content, max_segments=qa_per_doc)

        for idx, (page, segment) in enumerate(segments):
            prompt = build_prompt(few_shot_examples, segment)
            max_retries =2
            for retry in range(max_retries):
                try:
                    response = chat_model.chat(prompt)
                    parsed = parse_json(response)
                    if parsed is not None:
                        parsed["source_file"] = filename
                        parsed["page_num"] = page
                        parsed["content"] = segment
                        results.append(parsed)
                        break
                    else:
                        print(f"重试 {retry + 1}/{max_retries} - 文件 {filename} 第{idx + 1}段（页码：{page}）：无法解析 JSON")
                except Exception as e:
                    print(f"重试 {retry + 1}/{max_retries} - 文件 {filename} 第{idx + 1}段（页码：{page}）：{e}")
            else:
                print(f"错误 - 文件 {filename} 第{idx + 1}段（页码：{page}）：达到最大重试次数，无法获取有效数据")

    valid_results = []
    for result in results:
        question = result.get("question", "").strip()
        answer = result.get("answer", "").strip()
        if len(question) > 5 and len(answer) > 10:
            valid_results.append(result)

    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(valid_results, f, ensure_ascii=False, indent=2)

    print(f"\n成功保存 {len(valid_results)} 个高质量 QA 对到 {output_file}")


if __name__ == "__main__":
    main()


成功保存 30 个高质量 QA 对到 D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA6(4)(352)(334)豆包2.json


增加进度条，丰富提示词，统计数量，模型：deepseek-r1
报错后想到大模型计数能力不好，统计数量还是等QA对生成后再另外统计

In [13]:
import os
import json
import re
from openai import OpenAI
from tqdm import tqdm

class CHAT_MODEL:
    def __init__(self, api_key, base_url, model_name):
        self.llm = OpenAI(api_key=api_key, base_url=base_url)
        self.model_name = model_name

    def chat(self, user_prompt):
        completion = self.llm.chat.completions.create(
            model=self.model_name,
            messages=[{"role": "user", "content": user_prompt}],
        )
        return completion.choices[0].message.content


def extract_segments_with_page(md_text, max_segments=5, min_len=50):
    # 使用页码标题分割，如 ## 第3页
    page_splits = re.split(r'#+\s*第\s*(\d+)\s*页', md_text)
    segments = []

    if len(page_splits) <= 1:
        # 如果没有页码标识，按整段分
        raw_paragraphs = re.split(r'\n\s*\n', md_text)
        for para in raw_paragraphs:
            para = para.strip()
            if len(para) >= min_len:
                segments.append(("未知页码", para))
    else:
        for i in range(1, len(page_splits), 2):
            page = int(page_splits[i])
            # 跳过第 0 页和第 1 页
            if page in [0, 1,2]:
                continue
            content = page_splits[i + 1]
            paragraphs = re.split(r'\n\s*\n', content)
            for para in paragraphs:
                para = para.strip()
                if len(para) >= min_len:
                    segments.append((f"{page}", para))

    total = len(segments)
    if total <= max_segments:
        return segments
    step = total // max_segments
    return [segments[i] for i in range(0, total, step)][:max_segments]


def build_prompt(few_shot_examples, text_segment):
    prompt = (
        "你是一个资深保险条款分析师，任务是基于以下多个保险文档内容，生成用户可能会提出的业务相关问题，并结合文档信息提供专业、准确、结构清晰的回答。\n\n"
        "请根据以下文档内容，生成一个高质量的问题和答案对。问题应贴近实际业务场景，答案应清晰、信息丰富。\n\n"
        "### 核心要求（保留您原有标准）\n"
        "1. 问题必须模拟真实用户的业务需求或疑问，例如理赔流程、免责条款、适用场景、利益比较等，不是简单的信息回述或定义解释。\n"
        "2. 答案可以综合多个文档片段的内容，但必须确保准确，逻辑严谨，不得凭空捏造。\n"
        "3. 回答中如引用到不同文档，请明确列出引用的文档名称与页码。\n\n"
        "问题部分：围绕一个核心主题提出一个问题，语言表达应贴近用户常见问法，如：\n"
        "    - 什么是...？\n"
        "    - 是否可以说...？\n"
        "    - 请解释一下...的含义\n"
        "    - 如果...会怎样？\n"
        "    - 能否举个例子说明...？\n"
        "    - 为什么...会发生？\n"
        "    - ...的好处是什么？\n"
        "答案部分：逻辑清晰，内容完整。不得照抄原文，应体现理解和重组能力。\n\n"
        "### 新增规范\n"
        "1. 问题类型标注（必须三选一）：\n"
        "   ▸ 【是什么】定义解释类（如：什么是保单现金价值？）\n"
        "   ▸ 【为什么】原因分析类（如：为什么需要健康告知？）\n"
        "   ▸ 【怎么做】操作流程类（如：如何办理保单贷款？）\n\n"
        "2. 难度等级标注（必须三选一）：\n"
        "   ▸ 【简单】单概念问题（答案1-2句话）\n"
        "   ▸ 【中等】多要素问题（答案含步骤或条件）\n"
        "   ▸ 【困难】综合分析问题（需跨章节引用）\n\n"
        "### 输出格式（升级版）\n"
        "```json\n"
        "{\n"
        '  "question": "问题内容（使用客户自然语言表达）",\n'
        '  "answer": "答案内容",\n'
        '  "qa_type": "是什么/为什么/怎么做",\n'
        '  "difficulty": "简单/中等/困难",\n'
        '  "source_file": "文档名称",\n'
        '  "page_num": "页码",\n'
        '  "content": "引用的原始文本片段"\n'
        "}\n"
        "```\n\n"
        "### 参考案例\n"
        "【示例1-理赔场景】\n" + few_shot_examples[0] + "\n\n"
        "【示例2-条款解释】\n" + few_shot_examples[1] + "\n\n"
        "### 待分析内容\n"
        f"{text_segment}\n\n"
        "请生成符合所有要求的JSON输出："
    )
    return prompt


def parse_json(response):
    try:
        # 先尝试直接解析
        return json.loads(response)
    except json.JSONDecodeError:
        try:
            # 如果失败，尝试清理非法字符
            cleaned = re.sub(r'[\x00-\x1f\x7f-\x9f]', '', response)
            cleaned = re.sub(r'\\[^"\\/bfnrtu]', r'\\\\', cleaned)  # 修复非法转义
            return json.loads(cleaned)
        except json.JSONDecodeError as e:
            print(f"JSON解析失败（已尝试清理）: {str(e)[:100]}...")
            return None


def main():
    input_folder = r"D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\data\(7)"
    output_file = r"D:\火力全开的项目实践\宏利 pdf 文件\测试集\data\QA6(7).json"
    qa_per_doc = 10
    api_key = "c751d97c-ba64-4a9a-afad-a0db5b88dc37"
    base_url = "https://ark.cn-beijing.volces.com/api/v3"
    model_name = "deepseek-r1-distill-qwen-32b-250120"

    chat_model = CHAT_MODEL(api_key=api_key, base_url=base_url, model_name=model_name)

    few_shot_examples = [
        '''{
"question": "投资相连寿险保单过了宽限期还没缴费会怎样？",
"answer": "灵活投资宝/万利保障计划/宏宝保单将享有30日宽限期。若过后仍未缴费，客户会收到保单失效通知书，说明如何复效。",
"source_file": "电子行政运作手册_保单行政_2023_10.md",
"page_num": "6",
"content": "投資相連壽險計劃保單...將享有30日的寬限期..." }''',
        '''{
"question": "什么情况下保险顾问的营业积分会被扣回？",
"answer": "若某月结日的净已缴保费低于两个月前月结日的净已缴保费，保险顾问的积分将被扣回。",
"source_file": "电子行政运作手册_保单行政_2023_10.md",
"page_num": "7",
"content": "於第一年淨保費測試...若（N）月結日的淨已繳保費 ≤（N–2）月結日..." }'''
    ]

    results = []

    md_files = [f for f in os.listdir(input_folder) if f.endswith(".md")]
    
    for filename in tqdm(md_files, desc="📄 处理文档"):
        file_path = os.path.join(input_folder, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()

        segments = extract_segments_with_page(content, max_segments=qa_per_doc)

        for idx, (page, segment) in enumerate(tqdm(segments, desc=f"📝 {filename[:15]}...", leave=False)):
            prompt = build_prompt(few_shot_examples, segment)
            max_retries =2
            for retry in range(max_retries):
                try:
                    response = chat_model.chat(prompt)
                    parsed = parse_json(response)
                    if parsed is not None:
                        parsed["source_file"] = filename
                        parsed["page_num"] = page
                        parsed["content"] = segment
                        results.append(parsed)
                        break
                    else:
                        print(f"重试 {retry + 1}/{max_retries} - 文件 {filename} 第{idx + 1}段（页码：{page}）：无法解析 JSON")
                except Exception as e:
                    print(f"重试 {retry + 1}/{max_retries} - 文件 {filename} 第{idx + 1}段（页码：{page}）：{e}")
            else:
                print(f"错误 - 文件 {filename} 第{idx + 1}段（页码：{page}）：达到最大重试次数，无法获取有效数据")

    valid_results = []
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(valid_results, f, ensure_ascii=False, indent=2)

    # 新增难度统计功能
    if not valid_results:
        print("\n⚠️ 警告：未生成任何有效的QA对")
        print("可能原因：")
        print("- API返回数据格式不正确")
        print("- 文档内容不符合要求")
        print("- 网络或API连接问题")
        return

    difficulty_stats = {
        "简单": 0,
        "中等": 0,
        "困难": 0
    }
    
    qa_type_stats = {
        "是什么": 0,
        "为什么": 0,
        "怎么做": 0
    }

    for result in valid_results:
        difficulty_stats[result.get("difficulty", "")] += 1
        qa_type_stats[result.get("qa_type", "")] += 1

    print("\n" + "="*50)
    print("QA对生成统计报告:")
    print("-"*50)
    print("难度等级分布:")
    for level, count in difficulty_stats.items():
        print(f"  {level}: {count}个 ({count/len(valid_results)*100:.1f}%)")
    
    print("\n问题类型分布:")
    for qa_type, count in qa_type_stats.items():
        print(f"  {qa_type}: {count}个 ({count/len(valid_results)*100:.1f}%)")
    print("="*50)

    #print("\n质量检查:")
    #print(f"  有效QA对总数: {len(valid_results)}")
    #print(f"  平均每文档生成: {len(valid_results)/len(md_files):.1f}个QA对")
    #print("="*50)

    print(f"\n成功保存 {len(valid_results)} 个高质量 QA 对到 {output_file}")


if __name__ == "__main__":
    main()

📄 处理文档:   0%|          | 0/1 [00:00<?, ?it/s]

JSON解析失败（已尝试清理）: Expecting value: line 1 column 1 (char 0)...
重试 1/2 - 文件 守护无间危疾保 保单条款_2022_07.md 第1段（页码：3）：无法解析 JSON


JSON解析失败（已尝试清理）: Expecting value: line 1 column 1 (char 0)...
重试 2/2 - 文件 守护无间危疾保 保单条款_2022_07.md 第1段（页码：3）：无法解析 JSON
错误 - 文件 守护无间危疾保 保单条款_2022_07.md 第1段（页码：3）：达到最大重试次数，无法获取有效数据
JSON解析失败（已尝试清理）: Expecting value: line 1 column 1 (char 0)...
重试 1/2 - 文件 守护无间危疾保 保单条款_2022_07.md 第2段（页码：7）：无法解析 JSON


JSON解析失败（已尝试清理）: Expecting value: line 1 column 1 (char 0)...
重试 2/2 - 文件 守护无间危疾保 保单条款_2022_07.md 第2段（页码：7）：无法解析 JSON
错误 - 文件 守护无间危疾保 保单条款_2022_07.md 第2段（页码：7）：达到最大重试次数，无法获取有效数据
JSON解析失败（已尝试清理）: Expecting value: line 1 column 1 (char 0)...
重试 1/2 - 文件 守护无间危疾保 保单条款_2022_07.md 第3段（页码：11）：无法解析 JSON


JSON解析失败（已尝试清理）: Expecting value: line 1 column 1 (char 0)...
重试 2/2 - 文件 守护无间危疾保 保单条款_2022_07.md 第3段（页码：11）：无法解析 JSON
错误 - 文件 守护无间危疾保 保单条款_2022_07.md 第3段（页码：11）：达到最大重试次数，无法获取有效数据
JSON解析失败（已尝试清理）: Expecting value: line 1 column 1 (char 0)...
重试 1/2 - 文件 守护无间危疾保 保单条款_2022_07.md 第4段（页码：15）：无法解析 JSON


...

JSON解析失败（已尝试清理）: Expecting value: line 1 column 1 (char 0)...
重试 2/2 - 文件 守护无间危疾保 保单条款_2022_07.md 第7段（页码：31）：无法解析 JSON
错误 - 文件 守护无间危疾保 保单条款_2022_07.md 第7段（页码：31）：达到最大重试次数，无法获取有效数据
JSON解析失败（已尝试清理）: Expecting value: line 1 column 1 (char 0)...
重试 1/2 - 文件 守护无间危疾保 保单条款_2022_07.md 第8段（页码：34）：无法解析 JSON


JSON解析失败（已尝试清理）: Expecting value: line 1 column 1 (char 0)...
重试 2/2 - 文件 守护无间危疾保 保单条款_2022_07.md 第8段（页码：34）：无法解析 JSON
错误 - 文件 守护无间危疾保 保单条款_2022_07.md 第8段（页码：34）：达到最大重试次数，无法获取有效数据
JSON解析失败（已尝试清理）: Expecting value: line 1 column 1 (char 0)...
重试 1/2 - 文件 守护无间危疾保 保单条款_2022_07.md 第9段（页码：37）：无法解析 JSON


JSON解析失败（已尝试清理）: Expecting value: line 1 column 1 (char 0)...
重试 2/2 - 文件 守护无间危疾保 保单条款_2022_07.md 第9段（页码：37）：无法解析 JSON
错误 - 文件 守护无间危疾保 保单条款_2022_07.md 第9段（页码：37）：达到最大重试次数，无法获取有效数据
JSON解析失败（已尝试清理）: Expecting value: line 1 column 1 (char 0)...
重试 1/2 - 文件 守护无间危疾保 保单条款_2022_07.md 第10段（页码：41）：无法解析 JSON


📄 处理文档: 100%|██████████| 1/1 [14:56<00:00, 896.25s/it]

JSON解析失败（已尝试清理）: Expecting value: line 1 column 1 (char 0)...
重试 2/2 - 文件 守护无间危疾保 保单条款_2022_07.md 第10段（页码：41）：无法解析 JSON
错误 - 文件 守护无间危疾保 保单条款_2022_07.md 第10段（页码：41）：达到最大重试次数，无法获取有效数据

⚠️ 警告：未生成任何有效的QA对
可能原因：
- API返回数据格式不正确
- 文档内容不符合要求
- 网络或API连接问题
